### 스태킹(Stacking)

### 학습 내용

* 데이터 준비
* 1계층 모델
* 1계층 모델 평가
* 2계층 모델 
* 2계층 모델 평가

### 데이터 준비

In [1]:
# ---------------------------------
# 데이터 등의 사전 준비
# ----------------------------------
import numpy as np
import pandas as pd

# train_x는 학습 데이터, train_y는 목적 변수, test_x는 테스트 데이터
# pandas의 DataFrame, Series로 유지합니다.(numpy의 array로 유지하기도 합니다)

train = pd.read_csv('../input/sample-data/train_preprocessed.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
test_x = pd.read_csv('../input/sample-data/test_preprocessed.csv')

# neural net용의 데이터
train_nn = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x_nn = train_nn.drop(['target'], axis=1)
train_y_nn = train_nn['target']
test_x_nn = pd.read_csv('../input/sample-data/test_preprocessed_onehot.csv')

In [2]:
train.shape, train_x.shape, train_y.shape, test_x.shape

((10000, 29), (10000, 28), (10000,), (10000, 28))

In [3]:
train_nn.shape, train_x_nn.shape, train_y_nn.shape, test_x_nn.shape

((10000, 60), (10000, 59), (10000,), (10000, 59))

### 스태킹(stacking)

In [4]:
# ---------------------------------
# 스태킹(stacking)
# ----------------------------------
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

# models.py에 Model1Xgb, Model1NN, Model2Linear을 정의하는 것으로 함
# 각 클래스는 fit로 학습하고 predict로 예측값 확률을 출력

from models import Model1Xgb, Model1NN, Model2Linear

# 학습 데이터에 대한 ‘목적변수를 모르는’예측값과, 테스트 데이터에 대한 예측값을 반환하는 함수
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # 교차 검증으로 학습・예측하여 예측값과 인덱스를 보존
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # 검증에 대한 예측값을 연결하고 이후 원래 순서로 정렬
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # 테스트 데이터에 대한 예측값의 평균 획득
    preds_test = np.mean(preds_test, axis=0)
    return pred_train, preds_test

In [5]:
# 1계층 모델
# pred_train_1a, pred_train_1b는 학습 데이터의 검증에서의 예측값
# pred_test_1a, pred_test_1b는 테스트 데이터의 예측값
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, train_x, train_y, test_x)

[14:00:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-logloss:0.54088	eval-logloss:0.55003
[1]	train-logloss:0.45269	eval-logloss:0.47182
[2]	train-logloss:0.39482	eval-logloss:0.42026
[3]	train-logloss:0.35198	eval-logloss:0.38520
[4]	train-logloss:0.32021	eval-logloss:0.36150
[5]	train-logloss:0.29673	eval-logloss:0.34463
[6]	train-logloss:0.27610	eval-logloss:0.32900
[7]	train-logloss:0.25886	eval-logloss:0.31670
[8]	train-logloss:0.24363	eval-logloss:0.30775
[9]	train-logloss:0.23153	eval-logloss:0.30093
[14:00:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-logloss:0.53891	eval-logloss:0.54864
[1]	train-logloss:0.45219	eval-logloss:0.47149
[2]	train-logloss:0.39574	eval-logloss:0.419

In [6]:
model_1b = Model1NN()
pred_train_1b, pred_test_1b = predict_cv(model_1b, train_x_nn, train_y, test_x_nn)

Epoch 1/10
59/59 [==============================] - 1s 4ms/step - loss: 0.4259 - val_loss: 0.3907
Epoch 2/10
59/59 [==============================] - 0s 3ms/step - loss: 0.3624 - val_loss: 0.3819
Epoch 3/10
59/59 [==============================] - 0s 2ms/step - loss: 0.3401 - val_loss: 0.3686
Epoch 4/10
59/59 [==============================] - 0s 2ms/step - loss: 0.3204 - val_loss: 0.3617
Epoch 5/10
59/59 [==============================] - 0s 2ms/step - loss: 0.2970 - val_loss: 0.3489
Epoch 6/10
59/59 [==============================] - 0s 2ms/step - loss: 0.2645 - val_loss: 0.3345
Epoch 7/10
59/59 [==============================] - 0s 2ms/step - loss: 0.2405 - val_loss: 0.3167
Epoch 8/10
59/59 [==============================] - 0s 2ms/step - loss: 0.2141 - val_loss: 0.3055
Epoch 9/10
59/59 [==============================] - 0s 2ms/step - loss: 0.1979 - val_loss: 0.2922
Epoch 10/10
313/313 [==============================] - 0s 618us/step
Epoch 1/10
59/59 [==============================]

In [7]:
# 1계층 모델의 평가
print(f'logloss: {log_loss(train_y, pred_train_1a, eps=1e-7):.4f}')
print(f'logloss: {log_loss(train_y, pred_train_1b, eps=1e-7):.4f}')

logloss: 0.2967
logloss: 0.2784


### 2계층 모델

In [8]:
# 예측값을 특징으로 데이터 프레임을 작성
train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b})
test_x_2 = pd.DataFrame({'pred_1a': pred_test_1a, 'pred_1b': pred_test_1b})

# 2계층 모델
# pred_train_2는 2계층 모델의 학습 데이터로 교차 검증에서의 예측값
# pred_test_2는 2계층 모델의 테스트 데이터 예측값
model_2 = Model2Linear()
pred_train_2, pred_test_2 = predict_cv(model_2, train_x_2, train_y, test_x_2)
print(f'logloss: {log_loss(train_y, pred_train_2, eps=1e-7):.4f}')

logloss: 0.2487
